# Analyzing RCT data with Precision Adjustment

In [ ]:
install.packages("sandwich")
install.packages("lmtest")
install.packages("xtable")
install.packages("hdm")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
library(sandwich)
library(lmtest)
library(xtable)
library(hdm)

## Data

In this lab, we analyze the Pennsylvania re-employment bonus experiment, which was previously studied in "Sequential testing of duration data: the case of the Pennsylvania ‘reemployment bonus’ experiment" (Bilias, 2000), among others. These experiments were conducted in the 1980s by the U.S. Department of Labor to test the incentive effects of alternative compensation schemes for unemployment insurance (UI). In these experiments, UI claimants were randomly assigned either to a control group or one of five treatment groups. Actually, there are six treatment groups in the experiments. Here we focus on treatment group 4, but feel free to explore other treatment groups. In the control group the current rules of the UI applied. Individuals in the treatment groups were offered a cash bonus if they found a job within some pre-specified period of time (qualification period), provided that the job was retained for a specified duration. The treatments differed in the level of the bonus, the length of the qualification period, and whether the bonus was declining over time in the qualification period; see http://qed.econ.queensu.ca/jae/2000-v15.6/bilias/readme.b.txt for further details on data.
  

In [ ]:
## loading the data
file <- "https://raw.githubusercontent.com/CausalAIBook/MetricsMLNotebooks/main/data/penn_jae.dat"
Penn <- as.data.frame(read.table(file, header = TRUE))

n <- dim(Penn)[1]
p_1 <- dim(Penn)[2]
Penn <- subset(Penn, tg == 4 | tg == 0)
attach(Penn)

The following objects are masked from Penn (pos = 3):

    abdt, agegt54, agelt35, black, dep, durable, female, hispanic,
    husd, inuidur1, inuidur2, lusd, muld, nondurable, othrace, q1, q2,
    q3, q4, q5, q6, recall, tg




In [ ]:
T4 <- (tg == 4)
summary(T4)

   Mode   FALSE    TRUE 
logical    3354    1745 

In [ ]:
head(Penn)

,abdt,tg,inuidur1,inuidur2,female,black,hispanic,othrace,dep,q1,⋯,q5,q6,recall,agelt35,agegt54,durable,nondurable,lusd,husd,muld
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,10824,0,18,18,0,0,0,0,2,0,⋯,1,0,0,0,0,0,0,0,1,0
4,10824,0,1,1,0,0,0,0,0,0,⋯,1,0,0,0,0,0,0,1,0,0
5,10747,0,27,27,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,1,0,0
12,10607,4,9,9,0,0,0,0,0,0,⋯,0,0,0,1,0,0,0,0,0,1
13,10831,0,27,27,0,0,0,0,1,0,⋯,1,0,0,0,1,1,0,1,0,0
14,10845,0,27,27,1,0,0,0,0,0,⋯,1,0,0,0,1,0,0,1,0,0


### Model
To evaluate the impact of the treatments on unemployment duration, we consider the linear regression model:

$$
Y =  D \beta_1 + W'\beta_2 + \varepsilon, \quad E \varepsilon (D,W')' = 0,
$$

where $Y$ is  the  log of duration of unemployment, $D$ is a treatment  indicators,  and $W$ is a set of controls including age group dummies, gender, race, number of dependents, quarter of the experiment, location within the state, existence of recall expectations, and type of occupation.   Here $\beta_1$ is the ATE, if the RCT assumptions hold rigorously.


We also consider interactive regression model:

$$
Y =  D \alpha_1 + D W' \alpha_2 + W'\beta_2 + \varepsilon, \quad E \varepsilon (D,W', DW')' = 0,
$$
where $W$'s are demeaned (apart from the intercept), so that $\alpha_1$ is the ATE, if the RCT assumptions hold rigorously.

Under RCT, the projection coefficient $\beta_1$ has
the interpretation of the causal effect of the treatment on
the average outcome. We thus refer to $\beta_1$ as the average
treatment effect (ATE). Note that the covariates, here are
independent of the treatment $D$, so we can identify $\beta_1$ by
just linear regression of $Y$ on $D$, without adding covariates.
However we do add covariates in an effort to improve the
precision of our estimates of the average treatment effect.

### Analysis

We consider

*  classical 2-sample approach, no adjustment (CL)
*  classical linear regression adjustment (CRA)
*  interactive regression adjusment (IRA)

and carry out robust inference using the *estimatr* R packages.

# Carry out covariate balance check


We first look at the coefficients individually with a $t$-test, and then we adjust the $p$-values to control for family-wise error.

The regression below is done using "type='HC1'" which computes the correct Eicker-Huber-White standard errors, instead of the classical non-robust formula based on homoscedasticity.

In [ ]:
data <- model.matrix(T4 ~ (female + black + othrace + factor(dep) + q2 + q3 + q4 + q5 + q6 +
                             agelt35 + agegt54 + durable + lusd + husd)^2)

# individual t-tests
m <- lm(T4 ~ (female + black + othrace + factor(dep) + q2 + q3 + q4 + q5 + q6 +
                agelt35 + agegt54 + durable + lusd + husd)^2, data = as.data.frame(data))
coeftest(m, vcov = vcovHC(m, type = "HC1"))

Warning message in meatHC(x, type = type, omega = omega):
“HC1 covariances become (close to) singular if hat values are (close to) 1 as for observation(s) 3791”



t test of coefficients:

                        Estimate  Std. Error t value  Pr(>|t|)    
(Intercept)           0.32145725  0.16125607  1.9935 0.0462656 *  
female                0.10423328  0.13624779  0.7650 0.4442914    
black                 0.07164803  0.08288534  0.8644 0.3873969    
othrace               0.02801517  0.40496815  0.0692 0.9448502    
factor(dep)1         -0.07363340  0.20094637 -0.3664 0.7140574    
factor(dep)2         -0.10854072  0.15754307 -0.6890 0.4908810    
q2                   -0.02667937  0.16255836 -0.1641 0.8696419    
q3                   -0.00567387  0.16218178 -0.0350 0.9720934    
q4                    0.04334425  0.16233956  0.2670 0.7894821    
q5                    0.09386458  0.16157184  0.5809 0.5613028    
q6                   -0.22156423  0.15984049 -1.3862 0.1657604    
agelt35              -0.10923976  0.13323486 -0.8199 0.4123101    
agegt54              -0.43668630  0.13581268 -3.2154 0.0013111 ** 
durable              -0.12500967  0.

In [ ]:
# Create the model matrix using all two-way interactions among the controls
data <- model.matrix(T4 ~ (female + black + othrace + factor(dep) + q2 + q3 + q4 + q5 + q6 +
                             agelt35 + agegt54 + durable + lusd + husd)^2)

# Fit the linear model on the data matrix (treatment as outcome)
m <- lm(T4 ~ (female + black + othrace + factor(dep) + q2 + q3 + q4 + q5 + q6 +
                agelt35 + agegt54 + durable + lusd + husd)^2,
        data = as.data.frame(data))

# Compute robust standard errors using HC0 and HC1
result_HC0 <- coeftest(m, vcov = vcovHC(m, type = "HC0"))
result_HC1 <- coeftest(m, vcov = vcovHC(m, type = "HC1"))

# Combine the results into one table
results_table <- data.frame(
  Estimate = result_HC0[, 1],
  HC0_SE   = result_HC0[, 2],
  HC0_t    = result_HC0[, 3],
  HC0_p    = result_HC0[, 4],
  HC1_SE   = result_HC1[, 2],
  HC1_t    = result_HC1[, 3],
  HC1_p    = result_HC1[, 4]
)

# Name the rows using the coefficient names
rownames(results_table) <- rownames(result_HC0)

# Subset to only the first 10 rows (variables)
results_table_subset <- results_table[1:10, ]

# Load xtable package and print the subset table in LaTeX format
library(xtable)
print(xtable(results_table_subset, digits = c(0, 4, 4, 2, 4, 4, 2, 4)), type = "latex")


Warning message in meatHC(x, type = type, omega = omega):
“HC0 covariances become (close to) singular if hat values are (close to) 1 as for observation(s) 3791”
Warning message in meatHC(x, type = type, omega = omega):
“HC1 covariances become (close to) singular if hat values are (close to) 1 as for observation(s) 3791”


% latex table generated in R 4.4.2 by xtable 1.8-4 package
% Wed Feb 19 21:05:41 2025
\begin{table}[ht]
\centering
\begin{tabular}{rrrrrrrr}
  \hline
 & Estimate & HC0\_SE & HC0\_t & HC0\_p & HC1\_SE & HC1\_t & HC1\_p \\ 
  \hline
(Intercept) & 0.3215 & 0.1596 & 2.01 & 0.0441 & 0.1613 & 1.99 & 0.0463 \\ 
  female & 0.1042 & 0.1349 & 0.77 & 0.4396 & 0.1362 & 0.77 & 0.4443 \\ 
  black & 0.0716 & 0.0820 & 0.87 & 0.3825 & 0.0829 & 0.86 & 0.3874 \\ 
  othrace & 0.0280 & 0.4009 & 0.07 & 0.9443 & 0.4050 & 0.07 & 0.9449 \\ 
  factor(dep)1 & -0.0736 & 0.1989 & -0.37 & 0.7113 & 0.2009 & -0.37 & 0.7141 \\ 
  factor(dep)2 & -0.1085 & 0.1559 & -0.70 & 0.4864 & 0.1575 & -0.69 & 0.4909 \\ 
  q2 & -0.0267 & 0.1609 & -0.17 & 0.8683 & 0.1626 & -0.16 & 0.8696 \\ 
  q3 & -0.0057 & 0.1605 & -0.04 & 0.9718 & 0.1622 & -0.03 & 0.9721 \\ 
  q4 & 0.0433 & 0.1607 & 0.27 & 0.7874 & 0.1623 & 0.27 & 0.7895 \\ 
  q5 & 0.0939 & 0.1599 & 0.59 & 0.5573 & 0.1616 & 0.58 & 0.5613 \\ 
   \hline
\end{tabular}
\end{table}




To test balance conditions, we employ the Holm-Bonferroni step-down method. With 100+ hypotheses, the family-wise type I error, or the probability of making at least one type I error treating all hypotheses independently, is close to 1. To control for this, we adjust p-values with the following procedure.

First, set $\alpha=0.05$ and denote the list of $n$ p-values from the regression with the vector $p$.

1. Sort $p$ from smallest to largest, so $p_{(1)} \leq p_{(2)} \leq \cdots \leq p_{(n)}$. Denote the corresponding hypothesis for $p_{(i)}$ as $H_{(i)}$.
2. For $i=1,\ldots, n$,
- If $$p_{(i)} > \frac{\alpha}{n-i+1} $$ Break the loop and do not reject any $H_{(j)}$ for $j \geq i$.
- Else reject $H_{(i)}$ if $$p_{(i)} \leq \frac{\alpha}{n-i+1} $$ Increment $i := i+1$.





In [ ]:
holm_bonferroni <- function(p, alpha = 0.05) {
  n <- length(p)
  sig_beta <- c()

  for (i in 1:n) {
    if (sort(p)[i] > alpha / (n - i + 1)) {
      break
    } else {
      sig_beta <- c(sig_beta, order(p)[i])
    }
  }

  return(sig_beta)
}

p_values <- as.vector(coeftest(m, vcov = vcovHC(m, type = "HC1"))[, 4])
significant_indices <- holm_bonferroni(p_values, alpha = 0.05)
print(paste("Significant Coefficients (Indices): ", significant_indices))

p_values <- as.vector(coeftest(m, vcov = vcovHC(m, type = "HC0"))[, 4])
significant_indices <- holm_bonferroni(p_values, alpha = 0.05)
print(paste("Significant Coefficients (Indices): ", significant_indices))

Warning message in meatHC(x, type = type, omega = omega):
“HC1 covariances become (close to) singular if hat values are (close to) 1 as for observation(s) 3791”


[1] "Significant Coefficients (Indices):  84"
[2] "Significant Coefficients (Indices):  74"
[3] "Significant Coefficients (Indices):  94"
[4] "Significant Coefficients (Indices):  79"


Warning message in meatHC(x, type = type, omega = omega):
“HC0 covariances become (close to) singular if hat values are (close to) 1 as for observation(s) 3791”


[1] "Significant Coefficients (Indices):  84"
[2] "Significant Coefficients (Indices):  74"
[3] "Significant Coefficients (Indices):  94"
[4] "Significant Coefficients (Indices):  79"


There is also a built in R function to do this.

In [ ]:
p_values <- as.vector(coeftest(m, vcov = vcovHC(m, type = "HC1"))[, 4])
holm_reject <- p.adjust(sort(p_values), "holm") <= 0.05
holm_reject

Warning message in meatHC(x, type = type, omega = omega):
“HC1 covariances become (close to) singular if hat values are (close to) 1 as for observation(s) 3791”


[1]  TRUE  TRUE  TRUE  TRUE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [13] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [25] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [37] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [49] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [61] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [73] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [85] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [97] FALSE FALSE FALSE FALSE FALSE FALSE FALSE

We see that that even though this is a randomized experiment, balance

---

conditions are failed.
<!--
The holm method fails to reject any hypothesis. That is, we fail to reject the hypothesis that any coefficient is zero. Thus, in this randomized experiment, balance conditions are met. -->

# Model Specification

In [ ]:
# model specifications
# no adjustment (2-sample approach)
formula_cl <- log(inuidur1) ~ T4

# adding controls
formula_cra <- log(inuidur1) ~ T4 + (female + black + othrace + factor(dep) + q2 + q3 + q4 + q5 + q6 +
                                       agelt35 + agegt54 + durable + lusd + husd)^2
# Omitted dummies: q1, nondurable, muld

ols_cl <- lm(formula_cl)
ols_cra <- lm(formula_cra)

ols_cl <- coeftest(ols_cl, vcov = vcovHC(ols_cl, type = "HC1"))[2, ]
ols_cra <- coeftest(ols_cra, vcov = vcovHC(ols_cra, type = "HC1"))[2, ]

print(ols_cl)
print(ols_cra)

Warning message in meatHC(x, type = type, omega = omega):
“HC1 covariances become (close to) singular if hat values are (close to) 1 as for observation(s) 3791”


   Estimate  Std. Error     t value    Pr(>|t|) 
-0.08545541  0.03585569 -2.38331509  0.01719391 
   Estimate  Std. Error     t value    Pr(>|t|) 
-0.07968012  0.03559092 -2.23877661  0.02521432 


The interactive specificaiton corresponds to the approach introduced in Lin (2013).

In [ ]:
# interactive regression model;

X <- model.matrix(~ (female + black + othrace + factor(dep) + q2 + q3 + q4 + q5 + q6 +
                       agelt35 + agegt54 + durable + lusd + husd)^2)[, -1]
dim(X)

demean <- function(x) {
  x - mean(x)
}

X <- apply(X, 2, demean)
Y <- log(inuidur1)

ols_irm <- lm(Y ~ T4 * X)
ols_ira <- coeftest(ols_irm, vcov = vcovHC(ols_irm, type = "HC1"))[2, c(1:2)]

print(ols_ira)  # print out ATE estimate and conventional standard error.

# Because we de-meaned the covariates and interacted them with the treamtent
# we need to account for the error in the means of X, which has a first-order
# effect on the ATE estimate -- we need to add variance of CATE to
# the standard error scaled by n. Otherwise the se errors are valid for the
# sample average CATE.


# Create data frames for treated and control predictions
data_treated <- data.frame(T4 = TRUE, X)  # Simulate observations under treatment
data_control <- data.frame(T4 = FALSE, X)  # Simulate observations under control

# Compute predicted values under treatment and control
pred_treated <- predict(ols_irm, newdata = data_treated)
pred_control <- predict(ols_irm, newdata = data_control)

# Compute CATE as the difference
CATE <- pred_treated - pred_control
n <- length(Y)

# adjust the SE:
ols_ira[2] <- sqrt((ols_ira[2])^2 + var(CATE) / length(Y))
print(ols_ira)



[1] 5099  119

Warning message in meatHC(x, type = type, omega = omega):
“HC1 covariances become (close to) singular if hat values are (close to) 1 as for observation(s) 2433, 2543, 3791, 4889, 5087, 5090”


   Estimate  Std. Error 
-0.07550055  0.03560489 


Warning message in predict.lm(ols_irm, newdata = data_treated):
“prediction from rank-deficient fit; attr(*, "non-estim") has doubtful cases”
Warning message in predict.lm(ols_irm, newdata = data_control):
“prediction from rank-deficient fit; attr(*, "non-estim") has doubtful cases”


   Estimate  Std. Error 
-0.07550055  0.03600109 


Next we try out partialling out with lasso

In [ ]:
T4 <- demean(T4)

DX <- model.matrix(~ T4 * X)[, -1]

rlasso_ira <- summary(rlassoEffects(DX, log(inuidur1), index = 1, post = FALSE))$coef
print(rlasso_ira)

# correct standard error for demeaning

# Fit Lasso regression with treatment interaction
rlasso_model <- rlasso(log(inuidur1) ~ T4 * X, post = FALSE)

# Create data frames for treated and control predictions
data_treated <- data.frame(T4 = TRUE, X)  # Simulate observations under treatment
data_control <- data.frame(T4 = FALSE, X)  # Simulate observations under control

# Compute predicted values under treatment and control
pred_treated <- predict(rlasso_model, newdata = data_treated)
pred_control <- predict(rlasso_model, newdata = data_control)

# Compute CATE as the difference
CATE <- pred_treated - pred_control

# adjust the SE for demeaning

rlasso_ira[2] <- sqrt((rlasso_ira[2])^2 + var(CATE) / length(Y))

print(rlasso_ira)

     Estimate. Std. Error   t value   Pr(>|t|)
T4 -0.07976793 0.03542395 -2.251808 0.02433441


Warning message in colnames(mod.frame)[1] <- as.character(eval(object$call$formula)[[2]]):
“number of items to replace is not a multiple of replacement length”
Warning message in colnames(mod.frame)[1] <- as.character(eval(object$call$formula)[[2]]):
“number of items to replace is not a multiple of replacement length”


     Estimate. Std. Error   t value   Pr(>|t|)
T4 -0.07976793 0.03542463 -2.251808 0.02433441


### Results

In [ ]:
table <- matrix(0, 2, 4)
table[1, 1] <- ols_cl[1]
table[1, 2] <- ols_cra[1]
table[1, 3] <- ols_ira[1]
table[1, 4] <- rlasso_ira[1]

table[2, 1] <- ols_cl[2]
table[2, 2] <- ols_cra[2]
table[2, 3] <- ols_ira[2]
table[2, 4] <- rlasso_ira[2]


colnames(table) <- c("CL", "CRA", "IRA", "IRA w Lasso")
rownames(table) <- c("estimate", "standard error")
tab <- xtable(table, digits = 5)
tab

print(tab, type = "latex", digits = 5)

,CL,CRA,IRA,IRA w Lasso
,<dbl>,<dbl>,<dbl>,<dbl>
estimate,-0.08545541,-0.07968012,-0.07550055,-0.07976793
standard error,0.03585569,0.03559092,0.03600109,0.03542463


% latex table generated in R 4.4.2 by xtable 1.8-4 package
% Wed Feb 19 21:15:38 2025
\begin{table}[ht]
\centering
\begin{tabular}{rrrrr}
  \hline
 & CL & CRA & IRA & IRA w Lasso \\ 
  \hline
estimate & -0.08546 & -0.07968 & -0.07550 & -0.07977 \\ 
  standard error & 0.03586 & 0.03559 & 0.03600 & 0.03542 \\ 
   \hline
\end{tabular}
\end{table}


Treatment group 4 experiences an average decrease of about $7.8\%$ in the length of unemployment spell.


Observe that regression estimators delivers estimates that are slighly more efficient (lower standard errors) than the simple 2 mean estimator, but essentially all methods have very similar standard errors. From IRA results we also see that there is not any statistically detectable heterogeneity.  We also see the regression estimators offer slightly lower estimates -- these difference occur perhaps to due minor imbalance in the treatment allocation, which the regression estimators try to correct.




In [ ]:
# Load required packages
library(sandwich)
library(lmtest)
library(xtable)
library(hdm)

# ----------------------------------------------------
# 1. Baseline and Covariate-Adjusted Models (HC0)
# ----------------------------------------------------

# No adjustment (2-sample approach)
formula_cl <- log(inuidur1) ~ T4

# Adding controls (with all two-way interactions)
formula_cra <- log(inuidur1) ~ T4 + (female + black + othrace + factor(dep) +
                                       q2 + q3 + q4 + q5 + q6 +
                                       agelt35 + agegt54 + durable + lusd + husd)^2
# (Omitted dummies: q1, nondurable, muld)

# Fit the models
ols_cl_model <- lm(formula_cl)
ols_cra_model <- lm(formula_cra)

# Extract treatment coefficient (the second coefficient) with HC0 robust SEs
ols_cl <- coeftest(ols_cl_model, vcov = vcovHC(ols_cl_model, type = "HC0"))[2, ]
ols_cra <- coeftest(ols_cra_model, vcov = vcovHC(ols_cra_model, type = "HC0"))[2, ]

print("2-Sample (CL) Model (HC0):")
print(ols_cl)
print("Covariate-Adjusted (CRA) Model (HC0):")
print(ols_cra)

# ----------------------------------------------------
# 2. Interactive Regression Model (IRA) with HC0
# ----------------------------------------------------

# Create model matrix of controls with two-way interactions (drop intercept)
X <- model.matrix(~ (female + black + othrace + factor(dep) +
                      q2 + q3 + q4 + q5 + q6 +
                      agelt35 + agegt54 + durable + lusd + husd)^2)[, -1]
dim(X)  # (optional, to check dimensions)

# Define a function to demean variables
demean <- function(x) { x - mean(x) }

# Demean the covariates
X <- apply(X, 2, demean)
Y <- log(inuidur1)

# Fit interactive model: outcome regressed on T4 and its interactions with X
ols_irm <- lm(Y ~ T4 * X)

# Extract the coefficient on T4 (ATE) and its robust HC0 SE
ols_ira <- coeftest(ols_irm, vcov = vcovHC(ols_irm, type = "HC0"))[2, c(1:2)]
print("Interactive Regression (IRA) initial (HC0):")
print(ols_ira)

# For prediction: Create data frames for treated and control outcomes;
# ensure T4 is numeric (1 for treated, 0 for control)
data_treated <- data.frame(T4 = 1, X)
data_control <- data.frame(T4 = 0, X)

# Predict outcomes under treatment and control
pred_treated <- predict(ols_irm, newdata = data_treated)
pred_control <- predict(ols_irm, newdata = data_control)
CATE <- pred_treated - pred_control  # individual treatment effects
n <- length(Y)

# Adjust the SE to account for the error in demeaning X (variance of CATE / n)
ols_ira[2] <- sqrt((ols_ira[2])^2 + var(CATE) / n)
print("Interactive Regression (IRA) adjusted (HC0):")
print(ols_ira)

# ----------------------------------------------------
# 3. Lasso-Based Interactive Regression (IRA with Lasso) using HC0
# ----------------------------------------------------

# Demean the treatment variable T4 for the Lasso procedure
T4_demeaned <- demean(T4)

# Create model matrix for the interaction of T4_demeaned and X (drop intercept)
DX <- model.matrix(~ T4_demeaned * X)[, -1]

# Use rlassoEffects to estimate the treatment effect with Lasso (HC0 will be used in the summary)
rlasso_ira <- summary(rlassoEffects(DX, log(inuidur1), index = 1, post = FALSE))$coef
print("Lasso-based IRA initial (HC0):")
print(rlasso_ira)

# Fit the Lasso regression model with treatment interaction (non-post selection)
rlasso_model <- rlasso(log(inuidur1) ~ T4_demeaned * X, post = FALSE)

# For prediction: Create data frames for treated and control; ensure T4_demeaned is numeric
data_treated <- data.frame(T4_demeaned = 1, X)
data_control <- data.frame(T4_demeaned = 0, X)

# Predict outcomes under treatment and control using the Lasso model
pred_treated <- predict(rlasso_model, newdata = data_treated)
pred_control <- predict(rlasso_model, newdata = data_control)
CATE <- pred_treated - pred_control

# Adjust the Lasso-based SE for demeaning effects
rlasso_ira[2] <- sqrt((rlasso_ira[2])^2 + var(CATE) / n)
print("Lasso-based IRA adjusted (HC0):")
print(rlasso_ira)

# ----------------------------------------------------
# 4. Combine and Output Results in a LaTeX Table
# ----------------------------------------------------

# Create a matrix to hold the estimates and standard errors from all four models
result_table <- matrix(0, 2, 4)
result_table[1, 1] <- ols_cl[1]
result_table[1, 2] <- ols_cra[1]
result_table[1, 3] <- ols_ira[1]
result_table[1, 4] <- rlasso_ira[1]

result_table[2, 1] <- ols_cl[2]
result_table[2, 2] <- ols_cra[2]
result_table[2, 3] <- ols_ira[2]
result_table[2, 4] <- rlasso_ira[2]

colnames(result_table) <- c("CL", "CRA", "IRA", "IRA w Lasso")
rownames(result_table) <- c("Estimate", "Standard Error")

# Print the table in LaTeX format using xtable
tab <- xtable(result_table, digits = c(0, 5, 5, 5, 5))
print(tab, type = "latex", digits = 5)


Warning message in meatHC(x, type = type, omega = omega):
“HC0 covariances become (close to) singular if hat values are (close to) 1 as for observation(s) 3791”


[1] "2-Sample (CL) Model (HC0):"
   Estimate  Std. Error     t value    Pr(>|t|) 
-0.08545541  0.03584866 -2.38378264  0.01717211 
[1] "Covariate-Adjusted (CRA) Model (HC0):"
   Estimate  Std. Error     t value    Pr(>|t|) 
-0.07968012  0.03522609 -2.26196313  0.02374248 


[1] 5099  119

Warning message in meatHC(x, type = type, omega = omega):
“HC0 covariances become (close to) singular if hat values are (close to) 1 as for observation(s) 2433, 2543, 3791, 4889, 5087, 5090”


[1] "Interactive Regression (IRA) initial (HC0):"
   Estimate  Std. Error 
-0.07550055  0.03488895 


Warning message in predict.lm(ols_irm, newdata = data_treated):
“prediction from rank-deficient fit; attr(*, "non-estim") has doubtful cases”
Warning message in predict.lm(ols_irm, newdata = data_control):
“prediction from rank-deficient fit; attr(*, "non-estim") has doubtful cases”


[1] "Interactive Regression (IRA) adjusted (HC0):"
   Estimate  Std. Error 
-0.07550055  0.03529319 
[1] "Lasso-based IRA initial (HC0):"
              Estimate. Std. Error   t value   Pr(>|t|)
T4_demeaned -0.07976793 0.03542395 -2.251808 0.02433441


Warning message in colnames(mod.frame)[1] <- as.character(eval(object$call$formula)[[2]]):
“number of items to replace is not a multiple of replacement length”
Warning message in colnames(mod.frame)[1] <- as.character(eval(object$call$formula)[[2]]):
“number of items to replace is not a multiple of replacement length”


[1] "Lasso-based IRA adjusted (HC0):"
              Estimate. Std. Error   t value   Pr(>|t|)
T4_demeaned -0.07976793 0.03542463 -2.251808 0.02433441
% latex table generated in R 4.4.2 by xtable 1.8-4 package
% Wed Feb 19 21:20:06 2025
\begin{table}[ht]
\centering
\begin{tabular}{rrrrr}
  \hline
 & CL & CRA & IRA & IRA w Lasso \\ 
  \hline
Estimate & -0.08546 & -0.07968 & -0.07550 & -0.07977 \\ 
  Standard Error & 0.03585 & 0.03523 & 0.03529 & 0.03542 \\ 
   \hline
\end{tabular}
\end{table}


In [ ]:
# Load necessary packages
library(sandwich)
library(lmtest)
library(xtable)

# ---------------------------------------------
# 1. Flexible Controls Model (using HC1)
# ---------------------------------------------

# Create the model matrix using all two-way and three-way interactions among the controls
data <- model.matrix(T4 ~ (female + black + othrace + factor(dep) +
                             q2 + q3 + q4 + q5 + q6 +
                             agelt35 + agegt54 + durable + lusd + husd)^3)

# Fit the linear model on the data matrix (treatment as outcome)
m <- lm(T4 ~ (female + black + othrace + factor(dep) +
              q2 + q3 + q4 + q5 + q6 +
              agelt35 + agegt54 + durable + lusd + husd)^3,
        data = as.data.frame(data))

# Compute robust standard errors using HC1
result_HC1 <- coeftest(m, vcov = vcovHC(m, type = "HC1"))

# Combine the HC1 results into a table
results_table <- data.frame(
  Estimate = result_HC1[, 1],
  HC1_SE   = result_HC1[, 2],
  HC1_t    = result_HC1[, 3],
  HC1_p    = result_HC1[, 4]
)
rownames(results_table) <- rownames(result_HC1)

# Subset to only the first 10 rows (variables) for clarity
results_table_subset <- results_table[1:10, ]

# Print the subset table in LaTeX format using xtable
print(xtable(results_table_subset, digits = c(0, 4, 4, 2, 4)), type = "latex")


Warning message in meatHC(x, type = type, omega = omega):
“HC1 covariances become (close to) singular if hat values are (close to) 1 as for observation(s) 59, 73, 128, 135, 185, 249, 625, 1486, 1496, 1617, ...”


% latex table generated in R 4.4.2 by xtable 1.8-4 package
% Wed Feb 19 21:32:15 2025
\begin{table}[ht]
\centering
\begin{tabular}{rrrrr}
  \hline
 & Estimate & HC1\_SE & HC1\_t & HC1\_p \\ 
  \hline
(Intercept) & 0.0371 & 0.2948 & 0.13 & 0.8998 \\ 
  female & 0.1641 & 0.3217 & 0.51 & 0.6101 \\ 
  black & -0.0933 & 0.1261 & -0.74 & 0.4595 \\ 
  othrace & 0.8177 & 0.0968 & 8.45 & 0.0000 \\ 
  factor(dep)1 & -0.4260 & 0.6250 & -0.68 & 0.4955 \\ 
  factor(dep)2 & -0.0603 & 0.3593 & -0.17 & 0.8667 \\ 
  q2 & -0.1153 & 0.2986 & -0.39 & 0.6995 \\ 
  q3 & -0.0955 & 0.2986 & -0.32 & 0.7492 \\ 
  q4 & -0.0232 & 0.2995 & -0.08 & 0.9382 \\ 
  q5 & -0.0007 & 0.2981 & -0.00 & 0.9982 \\ 
   \hline
\end{tabular}
\end{table}


In [ ]:

# ---------------------------------------------
# 2. Balance Check Using Holm–Bonferroni (HC1)
# ---------------------------------------------

# Define the Holm–Bonferroni function
holm_bonferroni <- function(p, alpha = 0.05) {
  n <- length(p)
  sig_beta <- c()

  for (i in 1:n) {
    if (sort(p)[i] > alpha / (n - i + 1)) {
      break
    } else {
      sig_beta <- c(sig_beta, order(p)[i])
    }
  }

  return(sig_beta)
}

# Extract p-values from the HC1 results and apply the Holm–Bonferroni correction
p_values_HC1 <- as.vector(result_HC1[, 4])
significant_indices_HC1 <- holm_bonferroni(p_values_HC1, alpha = 0.05)
print(paste("Significant Coefficients (Indices) using HC1:",
            paste(significant_indices_HC1, collapse = ", ")))


[1] "Significant Coefficients (Indices) using HC1: 122, 51, 18, 45, 223, 229, 47, 120, 225, 227, 48, 219, 116, 4, 119, 117, 52, 199, 44, 222, 42, 216, 228, 221, 43"


In [ ]:
# --------------------------------------------------
# 1. Model Specifications: CL and CRA (using HC1) with Third-Order Interactions
# --------------------------------------------------

# (a) No adjustment (2-sample approach) remains unchanged:
formula_cl <- log(inuidur1) ~ T4

# (b) Adding controls with all two-way and three-way interactions
# (omitting some dummies: q1, nondurable, muld)
formula_cra <- log(inuidur1) ~ T4 + (female + black + othrace + factor(dep) +
                                       q2 + q3 + q4 + q5 + q6 +
                                       agelt35 + agegt54 + durable + lusd + husd)^3

# Fit models
ols_cl_model <- lm(formula_cl)
ols_cra_model <- lm(formula_cra)

# Extract the treatment coefficient (row 2) with HC1 robust standard errors
ols_cl <- coeftest(ols_cl_model, vcov = vcovHC(ols_cl_model, type = "HC1"))[2, ]
ols_cra <- coeftest(ols_cra_model, vcov = vcovHC(ols_cra_model, type = "HC1"))[2, ]

print("CL model (2-sample approach):")
print(ols_cl)
print("CRA model (covariate-adjusted, third-order interactions):")
print(ols_cra)

# --------------------------------------------------
# 2. Interactive Regression Model (IRA) using HC1 with Third-Order Interactions
# --------------------------------------------------

# Create model matrix for controls with all two-way and three-way interactions (drop intercept)
X <- model.matrix(~ (female + black + othrace + factor(dep) +
                      q2 + q3 + q4 + q5 + q6 +
                      agelt35 + agegt54 + durable + lusd + husd)^3)[, -1]
dim(X)  # Optional: view dimensions

# Function to demean a variable
demean <- function(x) {
  x - mean(x)
}

# Demean the covariates
X <- apply(X, 2, demean)
Y <- log(inuidur1)

# Fit the interactive model: outcome regressed on T4 and its interactions with X
ols_irm <- lm(Y ~ T4 * X)

# Extract the coefficient on T4 (ATE) using HC1 robust SEs (estimate & SE)
ols_ira <- coeftest(ols_irm, vcov = vcovHC(ols_irm, type = "HC1"))[2, c(1:2)]
print("Interactive Regression (IRA) initial (HC1):")
print(ols_ira)

# For prediction, ensure T4 is numeric (1 for treated, 0 for control)
data_treated <- data.frame(T4 = 1, X)
data_control <- data.frame(T4 = 0, X)

# Compute predicted outcomes under treatment and control
pred_treated <- predict(ols_irm, newdata = data_treated)
pred_control <- predict(ols_irm, newdata = data_control)
CATE <- pred_treated - pred_control  # Individual treatment effects
n <- length(Y)

# Adjust the standard error of the ATE by adding variance of CATE/n
ols_ira[2] <- sqrt((ols_ira[2])^2 + var(CATE) / n)
print("Interactive Regression (IRA) adjusted (HC1):")
print(ols_ira)

# --------------------------------------------------
# 3. Interactive Regression with Lasso (IRA w Lasso) using HC1 with Third-Order Interactions
# --------------------------------------------------

# For the Lasso procedure, demean T4 as well
T4_demeaned <- demean(T4)

# Create the model matrix for interactions between T4_demeaned and X (drop intercept)
DX <- model.matrix(~ T4_demeaned * X)[, -1]

# Estimate treatment effect using rlassoEffects (HC1 results provided in summary)
rlasso_ira <- summary(rlassoEffects(DX, log(inuidur1), index = 1, post = FALSE))$coef
print("Lasso-based Interactive Regression (initial) (HC1):")
print(rlasso_ira)

# Fit Lasso regression with treatment interaction (non-post-selection version)
rlasso_model <- rlasso(log(inuidur1) ~ T4_demeaned * X, post = FALSE)

# Create data for prediction (ensure T4_demeaned is numeric: 1 for treated, 0 for control)
data_treated <- data.frame(T4_demeaned = 1, X)
data_control <- data.frame(T4_demeaned = 0, X)

# Compute predicted values and individual treatment effects (CATE)
pred_treated <- predict(rlasso_model, newdata = data_treated)
pred_control <- predict(rlasso_model, newdata = data_control)
CATE <- pred_treated - pred_control

# Adjust the standard error for the Lasso-based ATE estimate
rlasso_ira[2] <- sqrt((rlasso_ira[2])^2 + var(CATE) / n)
print("Lasso-based Interactive Regression (adjusted) (HC1):")
print(rlasso_ira)

# --------------------------------------------------
# 4. Combine Results into a LaTeX Table
# --------------------------------------------------

result_table <- matrix(0, 2, 4)
result_table[1, 1] <- ols_cl[1]
result_table[1, 2] <- ols_cra[1]
result_table[1, 3] <- ols_ira[1]
result_table[1, 4] <- rlasso_ira[1]

result_table[2, 1] <- ols_cl[2]
result_table[2, 2] <- ols_cra[2]
result_table[2, 3] <- ols_ira[2]
result_table[2, 4] <- rlasso_ira[2]

colnames(result_table) <- c("CL", "CRA", "IRA", "IRA w Lasso")
rownames(result_table) <- c("estimate", "standard error")

tab <- xtable(result_table, digits = c(0, 5, 5, 5, 5))
print(tab, type = "latex", digits = 5)

Warning message in meatHC(x, type = type, omega = omega):
“HC1 covariances become (close to) singular if hat values are (close to) 1 as for observation(s) 59, 73, 128, 135, 185, 249, 625, 1486, 1496, 1617, ...”


[1] "CL model (2-sample approach):"
   Estimate  Std. Error     t value    Pr(>|t|) 
-0.08545541  0.03585569 -2.38331509  0.01719391 
[1] "CRA model (covariate-adjusted, third-order interactions):"
   Estimate  Std. Error     t value    Pr(>|t|) 
-0.08340662  0.03664070 -2.27633802  0.02286988 


[1] 5099  561

Warning message in meatHC(x, type = type, omega = omega):
“HC1 covariances become (close to) singular if hat values are (close to) 1 as for observation(s) 59, 73, 128, 135, 185, 249, 382, 481, 517, 625, ...”


[1] "Interactive Regression (IRA) initial (HC1):"
   Estimate  Std. Error 
-0.08379188  0.03703610 


Warning message in predict.lm(ols_irm, newdata = data_treated):
“prediction from rank-deficient fit; attr(*, "non-estim") has doubtful cases”
Warning message in predict.lm(ols_irm, newdata = data_control):
“prediction from rank-deficient fit; attr(*, "non-estim") has doubtful cases”


[1] "Interactive Regression (IRA) adjusted (HC1):"
   Estimate  Std. Error 
-0.08379188  0.03882384 
[1] "Lasso-based Interactive Regression (initial) (HC1):"
              Estimate. Std. Error   t value   Pr(>|t|)
T4_demeaned -0.08382729 0.03536386 -2.370422 0.01776778


Warning message in colnames(mod.frame)[1] <- as.character(eval(object$call$formula)[[2]]):
“number of items to replace is not a multiple of replacement length”
Warning message in colnames(mod.frame)[1] <- as.character(eval(object$call$formula)[[2]]):
“number of items to replace is not a multiple of replacement length”


[1] "Lasso-based Interactive Regression (adjusted) (HC1):"
              Estimate. Std. Error   t value   Pr(>|t|)
T4_demeaned -0.08382729 0.03537262 -2.370422 0.01776778
% latex table generated in R 4.4.2 by xtable 1.8-4 package
% Wed Feb 19 21:47:11 2025
\begin{table}[ht]
\centering
\begin{tabular}{rrrrr}
  \hline
 & CL & CRA & IRA & IRA w Lasso \\ 
  \hline
estimate & -0.08546 & -0.08341 & -0.08379 & -0.08383 \\ 
  standard error & 0.03586 & 0.03664 & 0.03882 & 0.03537 \\ 
   \hline
\end{tabular}
\end{table}
